In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import time
import os
import numpy as np
import random
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1177989684
import tensorflow as tf
from IPython import display

In [ ]:
dsm = np.array(Image.open('Urbis/UrbDSMY21_RBC_50cm/UrbDSMY21_RBC_50cm.tif'))
dtm = np.array(Image.open('Urbis/UrbDTMY21_RBC_50cm/UrbDTMY21_RBC_50cm.tif'))
rgb = np.array(Image.open('Urbis/50cm_NS_RGB/50cm_NS_RGB.tif'))

## Preparation of 512x512 and 256x256 patches from the full map
The patches (in the form of np ndarrays) are randomly split into train, test and val datasets <br>
For each patch, a version with and without the rgb channels is saved, as well as the dtm supervision.

In [ ]:
n=0
val = dsm[0][0]
y = 0

windows = []

for i in range(0, dsm.shape[0]-512,512):
    #print(i)
    for j in range(dsm.shape[1]):
        if y >= dsm.shape[1] - 512:
            y = 0
            break
        if not val in dsm[i:i+512,y:y+512]:
            windows.append((i,y))
            y += 512
            n+=1
        else : 
            y += 1
            
print(n)                  

In [ ]:
def list_dif(list1, list2):
    res = list1
    for item in list2:
        res.remove(item)
    return res

items = list(range(n))
train = random.sample(items, int(0.85*n))
items = list_dif(items, train)
test = random.sample(items, int(0.1*n))
val = list_dif(items, test)

In [ ]:
def save_images(images_list, sample, name):
    id512 = 1
    id256 = 1
    for item in sample:
        w = images_list[item]
        arr = dsm[w[0]:w[0]+512,w[1]:w[1]+512]
        arr_dtm = dtm[w[0]:w[0]+512,w[1]:w[1]+512]
        arr_rgb = np.concatenate((arr.reshape(arr.shape[0], arr.shape[1],1), rgb[w[0]:w[0]+512,w[1]:w[1]+512]), axis=2)
        np.save('np_dataset/512/' + name + '/' + str(id512), arr)
        np.save('np_dataset/512/' + name + '/rgb_' + str(id512), arr_rgb)
        np.save('np_dataset/512/' + name + '/dtm_' + str(id512), arr_dtm)
        id512+=1
        for i in range(2):
            for j in range(2):
                arr_256 = arr[i*256:(i+1)*256, j*256:(j+1)*256]
                arr_256_rgb = arr_rgb[i*256:(i+1)*256, j*256:(j+1)*256, :]
                arr_dtm_256 = arr_dtm[i*256:(i+1)*256, j*256:(j+1)*256]
                np.save('np_dataset/256/' + name + '/' + str(id256), arr_256)
                np.save('np_dataset/256/' + name + '/rgb_' + str(id256), arr_256_rgb)
                np.save('np_dataset/256/' + name + '/dtm_' + str(id256), arr_dtm_256)
                id256 += 1
        print(str(id512), end="\r")
        time.sleep(.05)
    return 0

In [ ]:
save_images(windows, train, 'train')
print('done train')
save_images(windows, test, 'test')
print('done test')
save_images(windows, val, 'val')

## Preparation of the tensorflow datasets

In [3]:
BATCH_SIZE = 1

In [4]:
def load(image_id, res, dataset='train', rgb = False):
    if rgb:
        input_image = np.load('np_dataset/' + res + '/'+ dataset + '/rgb_' + image_id + '.npy')
    else:
        input_image = np.load('np_dataset/' + res + '/'+ dataset + '/' + image_id + '.npy')
    real_image = np.load('np_dataset/' + res + '/'+ dataset + '/dtm_' + image_id + '.npy')
    return input_image, real_image

def normalize(dsm):
    dsm /= np.max(dsm)
    return dsm

def normalize_rgb(image):
    image = (image / 127.5) - 1
    return image   

def make_dataset(res, dataset='train', rgb = False):
    input_images = []
    real_images = []
    path = 'np_dataset/' + res + '/' + dataset
    id_list = [name[4:-4] for name in os.listdir(path) if 'dtm' in name]
    N = str(len(id_list))
    i = 0
    for num in id_list[:1200]:
        input_image, real_image = load(num, res, dataset, rgb)
        if rgb:
            input_image[:,:,1:] = normalize_rgb(input_image[:,:,1:])
            input_image[:,:,0] = normalize(input_image[:,:,0])
            real_image /= np.max(input_image[:,:,0])
        else:
            input_image = normalize(input_image)
            real_image /= np.max(input_image)
        input_images.append(input_image)
        real_images.append(real_image)
        i += 1
        print(str(i) + '/' + N, end='\r')
        time.sleep(.01)
    return np.array(input_images), np.array(real_images)

def reshape_tf(item, res, rgb):
    dsm, dtm = item[0], item[1]
    res = int(res)
    if not rgb: 
        dsm = tf.reshape(dsm, (res,res,1))
    return (dsm, tf.reshape(dtm, (res,res,1)))

In [6]:
def make_tf_dataset(res, dataset='train', rgb = False):
    input_ds, real_ds = make_dataset(res, dataset=dataset, rgb=rgb)
    tf_dataset = tf.data.Dataset.from_tensor_slices((input_ds, real_ds))
    tf_dataset = tf_dataset.map(lambda x, y: reshape_tf((x,y), res,rgb))
    tf_dataset = tf_dataset.batch(BATCH_SIZE)
    print(str(res) + ' ' + str(dataset) + ' ' + str(rgb) + ' done')
    return tf_dataset

In [7]:
#train_dataset_512 = make_tf_dataset('512')
#train_dataset_512.save('tf_dataset/train_dataset_512')

#test_dataset_512 = make_tf_dataset('512', dataset='test')
#test_dataset_512.save('tf_dataset/test_dataset_512')

#train_dataset_256 = make_tf_dataset('256')
#train_dataset_256.save('tf_dataset/train_dataset_256')

#test_dataset_256 = make_tf_dataset('256', dataset='test')
#test_dataset_256.save('tf_dataset/test_dataset_256')

#train_dataset_rgb_512 = make_tf_dataset('512', rgb = True)
#train_dataset_rgb_512.save('tf_dataset/train_dataset_rgb_512')

test_dataset_rgb_512 = make_tf_dataset('512', dataset='test', rgb = True)
test_dataset_rgb_512.save('tf_dataset/test_dataset_rgb_512')

#train_dataset_rgb_256 = make_tf_dataset('256', rgb = True)
#train_dataset_rgb_256.save('tf_dataset/train_dataset_rgb_256')

#test_dataset_rgb_256 = make_tf_dataset('256', dataset='test', rgb = True)
#test_dataset_rgb_256 = test_dataset_rgb_256.save('tf_dataset/test_dataset_rgb_256')

512 test True done


## Loading saved tf dataset

In [ ]:
current_ds = tf.data.Dataset.load('tf_dataset/train_dataset_256_rgb')

In [ ]:
example = [item for item in current_ds.take(1)]
example[0]

In [ ]:
type(reshape_tf(example[0], '256', False))

Error in the shape of the images in the dataset --> instead of remaking the dataset they are loaded and reshaped

In [ ]:
example = [item for item in current_ds.take(1)]
inp, re = example[0][0], example[0][1]
inp

In [ ]:
for item in current_ds.take(6):
    plt.figure(figsize=(5,5))
    plt.subplot(1,2,1)
    plt.imshow(item[0][0])
    plt.subplot(1,2,2)
    plt.imshow(item[1][0])

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))